# VOLTTRON Aggregator Notebook

This notebook sets up and executes aggregation of forwarded data
from other VOLTTRON instances, using a historian to record the data.

Most of the notebook's setup and execution is done with shell commands, called from Python.

# Setup: Prepare the Volttron Environment

VOLTTRON must be installed before using this notebook. For detailed instructions on
installing and configuring a VOLTTRON/Jupyter server environment, see [Jupyter Notebooks](http://volttron.readthedocs.io/en/devguides/supporting/utilities/JupyterNotebooks.html) 
in VOLTTRON ReadTheDocs.

As is described in that guide, environment variables should have been defined before starting 
the Jupyter server:

````
$ export VOLTTRON_ROOT=~/repos/volttron
````
        (path of the VOLTTRON repository, installed prior to running bootstrap)

````
$ export VOLTTRON_HOME=~/.volttron
````
        (directory in which the VOLTTRON instance runs)

The first VOLTTRON instance on a server usually runs, by convention, in ~/.volttron.
If multiple VOLTTRON instances are to be run on a single host, each must have its own VOLTTRON_HOME.

Also before starting the Jupyter server, a VOLTTRON virtual environment should have been 
activated by executing the following in $VOLTTRON_ROOT:

````
$ source env/bin/activate
````

The Python code below does some initialization to prepare for the steps that follow.

In [ ]:
import datetime
import json
import os
import pprint
import sqlite3
import subprocess
import sys
import time

# Define a "run this shell command" method, wrapping subprocess.check_output()
def _sh(shell_command, shell=True, stderr=None):
    try:
        return_value = subprocess.check_output(shell_command, shell=shell, stderr=stderr)
    except Exception, err:
        print('Shell command failed: {}', shell_command)
        print(err)
        return_value = 'Error'
    return return_value

# Same as _sh(), except that this also prints the command output, preceded by an optional label.
def _print_sh(shell_command, label=None, **kwargs):
    print('{0}: {1}\n'.format(label+':' if label else '', _sh(shell_command, **kwargs)))

# Set up local variables vhome and vroot.
# The environment variables VOLTTRON_ROOT and VOLTTRON_HOME should already be defined -- see above.
vroot = %env VOLTTRON_ROOT
vhome = %env VOLTTRON_HOME
print("VOLTTRON_ROOT={}".format(vroot))
print("VOLTTRON_HOME={}".format(vhome))

# Define a VIP_SOCKET environment variable for use while installing and running agents.
socket_name = 'ipc://' + vhome + '/run/vip.socket'
%env VIP_SOCKET=$socket_name

# Run from the VOLTTRON root directory.
os.chdir(vroot)

data_dir = vhome + '/data'

print("Initialization complete")

# Setup: Shut Down All Agents

This ensures a clean agent installation process by the notebook.

In [ ]:
print('Wait for the list to be displayed, and confirm that no agents are listed as running...\n')

# Shut down all agents.
_sh('volttron-ctl shutdown')

# List agent status to verify that the status of each agent is 0 or blank.
_print_sh('volttron-ctl status', stderr=subprocess.STDOUT)

# Setup: Discover the Aggregator's Network Parameters

Each Collector and Observer must know this Aggregator's network parameters
so that it can forward data to it or pull data from it.
Discover those parameters now.

Copy the vip-address's IP and port, and the serverkey,
to the Collector or Observer notebook under
'Setup: Configure the Aggregator's Network Parameters'.

Also, make sure that this port is open for TCP access on the Aggregator's host.

In [ ]:
# Obtain this server's IP address, volttron port number (usually 22916), and server key:
print('Obtaining network parameters and server key; please wait...\n')
_print_sh('curl ifconfig.me', label='Public IP address')
_print_sh('volttron-ctl auth serverkey', label='Serverkey')
_print_sh('cat {}/config'.format(vhome), label='Config file')

# Setup: Add Each Collector and Observer to known_hosts

Do the following for each Collector and Observer, substituting that server's
IP address, port number and server key below.

Get each Collector's data from 'Setup: Update the Aggregator's known_hosts File' 
in the Collector notebook.

Get each Observer's data from 'Setup: Update the Aggregator's known_hosts File' 
in the Observer notebook.

In [ ]:
# For each remote VOLTTRON instance (Collector or Observer),
# change the following three strings and then execute this code
# to add the instance's properties to the Aggregator's known_hosts file:
remote_ip_address = 'foo'
remote_port_number = '22916'
remote_server_key = 'bar'

_sh('volttron-ctl auth add-known-host --host {0}:{1} --serverkey {2}'.format(remote_ip_address, 
                                                                             remote_port_number, 
                                                                             remote_server_key))
with open(vhome + '/known_hosts', 'r') as kh_file:
    print 'known_hosts file contents are now: \n{0}'.format(kh_file.read())

# Setup: Configure a SQLHistorian

In [ ]:
# Create a SQLHistorian configuration specifically for this project

# The historian's database will reside in $VOLTTRON_HOME/data. 
# Make sure that the directory exists.
if not os.path.exists(data_dir):
    _sh('mkdir {0}'.format(data_dir))

config = '''{{
    "agentid": "sqlhistorian-sqlite",
    "connection": {{
        "type": "sqlite",
        "params": {{
            "database": "{0}/historian.sqlite"
        }}
    }},
    "tables_def": {{
        "table_prefix": "",
        "data_table": "data_table",
        "topics_table": "topics_table",
        "meta_table": "meta_table"
    }}
}}'''.format(data_dir)
print("config = {}".format(config))
config_path = vhome + '/my_aggregator_historian.config'
with open(config_path, 'w') as file:
    file.write(config)
print('Historian configuration written to {}'.format(config_path))

# Setup: Install Agents

Install each agent employed by the Aggregator: a SQLHistorian and 2 Volttron Central agents.

In [ ]:
print('Wait for the list to be displayed, then confirm that all of these agents appear in it...')

def install_agent(dir=None, id=None, config=None, tag=None):
    script_install_command = 'python scripts/install-agent.py -s {0} -i {1} -c {2} -t {3} -f'
    _sh(script_install_command.format(dir, id, config, tag))
    print('Installed {}'.format(tag))

# Install a SQL Historian agent that captures metrics in a SQLite database
install_agent(dir=vroot+'/services/core/SQLHistorian',
              id='sqlite_historian',
              config=vhome + '/my_aggregator_historian.config',
              tag='sqlite_historian')

# Install a Platform Agent
install_agent(dir=vroot+'/services/core/VolttronCentralPlatform',
              id='platform.agent',
              config=vroot+'/services/core/VolttronCentralPlatform/config', 
              tag='vcp')

# Install a Volttron Central Agent
install_agent(dir=vroot+'/services/core/VolttronCentral',
              id='volttron.central',
              config=vroot+'/services/core/VolttronCentral/config', 
              tag='vc')

# List agent status to verify that the agents were installed successfully.
_print_sh('volttron-ctl status', stderr=subprocess.STDOUT)

# Execution: Refresh variables, stop agents, delete database

In [ ]:
print('Make a fresh start - refresh variable definitions, shut down any running agents, refresh the database')

import datetime
import json
import os
import pprint
import sqlite3
import subprocess
import sys
import time

# Define a "run this shell command" method, wrapping subprocess.check_output()
def _sh(shell_command, shell=True, stderr=None):
    try:
        return_value = subprocess.check_output(shell_command, shell=shell, stderr=stderr)
    except Exception, err:
        print('Shell command failed: {}', shell_command)
        print(err)
        return_value = 'Error'
    return return_value

# Same as _sh(), except that this also prints the command output, preceded by an optional label.
def _print_sh(shell_command, label=None, **kwargs):
    print('{0}: {1}\n'.format(label+':' if label else '', _sh(shell_command, **kwargs)))

# Set up local variables vhome and vroot.
# The environment variables VOLTTRON_ROOT and VOLTTRON_HOME should already be defined -- see above.
vroot = %env VOLTTRON_ROOT
vhome = %env VOLTTRON_HOME
print("VOLTTRON_ROOT={}".format(vroot))
print("VOLTTRON_HOME={}".format(vhome))

# Define a VIP_SOCKET environment variable for use while installing and running agents.
socket_name = 'ipc://' + vhome + '/run/vip.socket'
%env VIP_SOCKET=$socket_name

# Run from the VOLTTRON root directory.
os.chdir(vroot)

data_dir = vhome + '/data'

# Delete the Historian's SQLite database to get a fresh start.
if os.path.exists(data_dir + '/historian.sqlite'):
    _sh('rm {0}'.format(data_dir + '/historian.sqlite'))

# Shut down all agents.
_sh('volttron-ctl shutdown')

# List agent status to verify that the status of each agent is 0 or blank.
_print_sh('volttron-ctl status', stderr=subprocess.STDOUT)

# Execution: Start the agents

When ready to start collecting metrics from Collectors, using the Historian to record them,  start the agents.

In [ ]:
print('Wait for the list to be displayed, then confirm that each started agent is running...')

_sh('volttron-ctl start --tag sqlite_historian')
_sh('volttron-ctl start --tag vcp')
_sh('volttron-ctl start --tag vc')

# List agent status to verify that the started agents have status "running".
_print_sh('volttron-ctl status', stderr=subprocess.STDOUT)

# Data Reporting: Prepare to Execute Sqlite Commands

In [ ]:
# Define a function that executes SQLite commands on our Historian database
def run_sqlite_cmd(command_string):
    _print_sh('sqlite3 {0} {1}'.format(data_dir + '/historian.sqlite', command_string))

# Data Reporting: Describe the Historian's Database Schema

Start the data reporting process by displaying the schema of the Historian's SQLite database.

In [ ]:
run_sqlite_cmd('".schema"')

# Data Reporting: List the Topics

List each topic in the database's topics_table. This is the list of each type of data that has been captured and stored.

In [ ]:
run_sqlite_cmd('"SELECT * FROM topics_table;"')

# Data Reporting: List Values for a Single Topic

Select a single topic by name, and list each value in the database for it.

In [ ]:
topic_name = 'simstorage/soc_kwh'

display_variables = 'ts, value_string'
join_statement = 'INNER JOIN topics_table on (data_table.topic_id = topics_table.topic_id) '
sqlite_cmd = '''"SELECT {0} FROM data_table {1} WHERE topics_table.topic_name = '{2}';"'''.format(
    display_variables,
    join_statement,
    topic_name)
print('sqlite command: \n{0}\n'.format(sqlite_cmd))

run_sqlite_cmd(sqlite_cmd)

# Data Reporting: Graph Values for a Single Topic

Use numpy and matplotlib to produce a graph of the values for a topic.

In [ ]:
topic_name = 'simstorage/soc_kwh'

display_variables = 'ts, value_string'
join_statement = 'INNER JOIN topics_table on (data_table.topic_id = topics_table.topic_id) '
sqlite_cmd = '''SELECT {0} FROM data_table {1} WHERE topics_table.topic_name = '{2}';'''.format(
    display_variables,
    join_statement,
    topic_name)
print('sqlite command: \n{0}\n'.format(sqlite_cmd))

import numpy
import matplotlib.pyplot as plt
from matplotlib import dates

# Connect to the SQLite database
conn = sqlite3.connect(data_dir + '/historian.sqlite')
c = conn.cursor()

# Populate graphArray with the result of querying the database for the specified topic.
graphArray = []
for row in c.execute(sqlite_cmd):
    # Remove parentheses and single quotes
    row_string_filtered = str(row).translate(None, "()'u\'")
    # In Python 3, the Unicode string would need to be filtered like this:
    # row_string_filtered = str(row).translate({ord(c): None for c in "()'u\'"})
    graphArray.append(row_string_filtered)

if graphArray:
    timestamps, values = numpy.loadtxt(graphArray,
                                       delimiter=',',
                                       unpack=True,
                                       converters={0: dates.strpdate2num('%Y-%m-%dT%H:%M:%S.%f+00:00')})
    fig = plt.figure()
    fig.add_subplot(1, 1, 1, facecolor='white')
    plt.plot_date(x=timestamps, y=values, fmt='b-')
    plt.gcf().autofmt_xdate()
    plt.show()
else:
    print('No data returned from query')

# Shutdown: Stop all agents

When finished, stop all VOLTTRON agents.

In [ ]:
# Stop all agents.
_sh('volttron-ctl shutdown')

# Verify that all agents have been stopped.
_print_sh('volttron-ctl status', stderr=subprocess.STDOUT)